get minst dataset

 |

 |

\ /

add + - / *  = signs

 |

 |

\ /

train clasifyer

 |

 |

\ /

profit

In [ ]:
from PIL import Image, ImageFilter
import numpy as np
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from math import floor
import PIL.ImageOps

tf.enable_v2_behavior()

In [ ]:
def test_my_image(image_path, model, s = 0.1):
  oi = Image.open(image_path)
  map = np.zeros((oi.size))
  mm = []
  connections = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]
  def bfs(start):
    stack = [start]
    minn = start
    maxx = start
    i = 0
    while (len(stack) != 0):
      i = i + 1
      cur = stack.pop()
      map[cur] = 1
      minn = (min(minn[0], cur[0]), min(minn[1], cur[1]))
      maxx = (max(maxx[0], cur[0]), max(maxx[1], cur[1]))
      for a in connections:
        newxy = (cur[0] + a[0], cur[1] + a[1])
        if (newxy[0] > 0 and newxy[0] < oi.size[0] and newxy[1] > 0 and newxy[1] < oi.size[1] and map[newxy] == 0):
          pix = oi.getpixel(newxy)
          map[newxy] = 1
          norm = np.linalg.norm(pix)
          if (norm < 50.0):
            stack.append(newxy)
    return [(minn[0] - 1, minn[1] - 1), (maxx[0] + 1, maxx[1] + 1)]
  
  def modify(img):
    slack = [30, 30]
    szx, szy = (img[1][0] - img[0][0], img[1][1] - img[0][1])
    if (szx > szy):
      slack[0] = floor(szx * s)
      slack[1] = floor(slack[0] * szx / szy)
    else:
      slack[1] = floor(szy * s)
      slack[0] = floor(slack[0] * szy / szx)  
    return PIL.ImageOps.invert(oi.crop((img[0][0] - slack[0], img[0][1] - slack[1], img[1][0] + slack[0], img[1][1] + slack[1])))

  for i in range(oi.size[0]):
    for j in range(oi.size[1]):
      pix = oi.getpixel((i, j))
      norm = np.linalg.norm(pix)
      if (map[i, j] == 0 and norm < 50.0):
        mm.append(bfs((i, j)))
      map[i, j] = 1

  ii = [modify(mm[i]) for i in range(len(mm))]
  iiresized = [ii[i].resize((28, 28)) for i in range(len(ii))]
  iirs = [((np.array(ii[i].resize((28, 28)))).reshape(1, 28, 28, 3)[:,:,:, 0].reshape(1, 28, 28, 1) / 255.) for i in range(len(ii))]
  for i in range(len(ii)):
    arr = iirs[i]
    p = np.exp(model.predict(arr)[0])
    pred = p / np.linalg.norm(p)
    print((np.max(pred), np.argmax(pred), pred))
  return iiresized

In [ ]:
(train_ds, test_ds), info_ds = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
train_ds

<_OptionsDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>

In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label


In [ ]:

train_ds = train_ds.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE). \
      cache(). \
      shuffle(info_ds.splits['train'].num_examples). \
      batch(2**13). \
      prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
test_ds = test_ds.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE) \
  .cache() \
  .batch(2**13) \
  .prefetch(tf.data.experimental.AUTOTUNE) 

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(50, 5, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(50, 5, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(2**6, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
model.fit(
    train_ds,
    epochs=120,
    validation_data=test_ds
)
model.summary()

Epoch 1/120
8/8 [==============================] - 15s 379ms/step - loss: 2.1352 - sparse_categorical_accuracy: 0.3695 - val_loss: 1.3335 - val_sparse_categorical_accuracy: 0.7578
Epoch 2/120
8/8 [==============================] - 1s 144ms/step - loss: 1.0736 - sparse_categorical_accuracy: 0.7851 - val_loss: 0.4793 - val_sparse_categorical_accuracy: 0.8601
Epoch 3/120
8/8 [==============================] - 1s 159ms/step - loss: 0.4564 - sparse_categorical_accuracy: 0.8628 - val_loss: 0.3409 - val_sparse_categorical_accuracy: 0.9014
Epoch 4/120
8/8 [==============================] - 1s 143ms/step - loss: 0.3512 - sparse_categorical_accuracy: 0.8975 - val_loss: 0.2830 - val_sparse_categorical_accuracy: 0.9176
Epoch 5/120
8/8 [==============================] - 1s 144ms/step - loss: 0.2870 - sparse_categorical_accuracy: 0.9157 - val_loss: 0.2324 - val_sparse_categorical_accuracy: 0.9296
Epoch 6/120
8/8 [==============================] - 1s 144ms/step - loss: 0.2380 - sparse_categorical_acc

In [ ]:
ims = test_my_image("zzz.bmp", model, 0.0)

(0.97476405, 7, array([8.9000869e-14, 2.8326782e-05, 2.2721923e-11, 4.2474326e-08,
       2.2114407e-01, 4.7592836e-04, 4.2197920e-10, 9.7476405e-01,
       3.0179478e-06, 3.0498138e-02], dtype=float32))
(1.0, 1, array([1.2763681e-11, 1.0000000e+00, 5.0113980e-12, 8.8887665e-13,
       8.3065293e-05, 1.8877236e-10, 1.9865554e-09, 3.9149336e-06,
       1.0843193e-07, 5.1600796e-10], dtype=float32))
(0.9987176, 5, array([1.5250836e-09, 5.1685632e-03, 2.5593024e-06, 6.5716956e-04,
       1.0944335e-07, 9.9871761e-01, 7.3056822e-10, 4.8405956e-02,
       5.4198492e-05, 1.3888696e-02], dtype=float32))
(0.99999994, 1, array([4.4997500e-06, 9.9999994e-01, 1.0543959e-04, 2.7318238e-05,
       5.5028354e-06, 4.7486105e-06, 2.0182239e-07, 2.0042992e-04,
       7.4848482e-05, 2.8830536e-06], dtype=float32))
(1.0, 6, array([2.3234919e-12, 1.0103472e-11, 2.9169191e-11, 3.2101727e-15,
       6.6443552e-13, 3.3278823e-07, 1.0000000e+00, 1.3090948e-15,
       6.1719265e-06, 6.5614037e-17], dtype=float

In [ ]:
ims[0]

In [ ]:
t1 = test_ds.take(1)
i = i + 1
Image.fromarray(list(t1.as_numpy_iterator())[0][0][i].reshape(28, 28)* 255).convert("RGB")


In [ ]:
class_names

['%',
 '*',
 '+',
 '-',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '[',
 ']']

In [ ]:
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle/kaggle.json

In [ ]:
import kaggle

In [ ]:
!kaggle datasets download -d michelheusser/handwritten-digits-and-operators

100% 205M/205M [00:01<00:00, 121MB/s]
100% 205M/205M [00:01<00:00, 112MB/s]


In [ ]:
!unzip handwritten-digits-and-operators.zip > /dev/null

In [ ]:
data_dir = "/content/CompleteImages/All data (Compressed)"
img_height = 28
img_width = 28
batch_size = 2**13

In [ ]:
train_ds_2 = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  batch_size=1,
  image_size=(img_height, img_width))
val_ds_2 = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  batch_size=1,
  image_size=(img_height, img_width))
     

train_ds_2 = train_ds_2 \
      .map(lambda x, y: (255 - tf.reshape(x[:, :, :, 0], [28, 28, 1]), y), num_parallel_calls=tf.data.experimental.AUTOTUNE) \
      .map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE) \
      .cache() \
      .batch(2**14) \
      .prefetch(tf.data.experimental.AUTOTUNE)
val_ds_2 = val_ds_2 \
      .map(lambda x, y: (255 - tf.reshape(x[:, :, :, 0], [28, 28, 1]), y), num_parallel_calls=tf.data.experimental.AUTOTUNE) \
      .map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE) \
      .cache() \
      .batch(2**14) \
      .prefetch(tf.data.experimental.AUTOTUNE)

Found 333895 files belonging to 16 classes.
Using 267116 files for training.
Found 333895 files belonging to 16 classes.
Using 66779 files for validation.


In [ ]:
t11 = train_ds_2.take(1)
t11iter = t11.as_numpy_iterator();
t11iter.next()[0].shape

(16384, 28, 28, 1)

In [ ]:
num_classes = 16

model2 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(2, 5, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(2, 5, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])
model2.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
model2.fit(
    train_ds_2,
    epochs=2000,
    validation_data=val_ds_2
)
model2.summary()

Epoch 1/2000
17/17 [==============================] - 1s 57ms/step - loss: 2.7929 - sparse_categorical_accuracy: 0.0810 - val_loss: 2.7635 - val_sparse_categorical_accuracy: 0.0782
Epoch 2/2000
17/17 [==============================] - 1s 48ms/step - loss: 2.7593 - sparse_categorical_accuracy: 0.0802 - val_loss: 2.7448 - val_sparse_categorical_accuracy: 0.0922
Epoch 3/2000
17/17 [==============================] - 1s 49ms/step - loss: 2.7357 - sparse_categorical_accuracy: 0.0992 - val_loss: 2.6965 - val_sparse_categorical_accuracy: 0.1209
Epoch 4/2000
17/17 [==============================] - 1s 48ms/step - loss: 2.6718 - sparse_categorical_accuracy: 0.1349 - val_loss: 2.5691 - val_sparse_categorical_accuracy: 0.1783
Epoch 5/2000
17/17 [==============================] - 1s 49ms/step - loss: 2.5176 - sparse_categorical_accuracy: 0.2012 - val_loss: 2.3389 - val_sparse_categorical_accuracy: 0.2807
Epoch 6/2000
17/17 [==============================] - 1s 49ms/step - loss: 2.2801 - sparse_cate

In [ ]:
ims = test_my_image("txt2.bmp", model2, 0.1)

(0.99932903, 0, array([9.9932903e-01, 6.5601824e-10, 8.8110603e-03, 1.9387317e-05,
       1.0739258e-09, 3.3974051e-02, 5.4494467e-06, 1.4763081e-06,
       3.4718356e-05, 6.7074889e-06, 3.4989746e-08, 6.4541539e-04,
       9.9988773e-10, 9.2823593e-06, 2.6393614e-03, 1.0111131e-02],
      dtype=float32))
(0.94851404, 0, array([9.4851404e-01, 1.6822291e-16, 8.9380082e-06, 9.5268644e-02,
       5.3423533e-05, 8.0238395e-05, 8.6970869e-07, 1.1450356e-11,
       9.6676558e-08, 9.1039766e-08, 4.7573323e-09, 1.9502453e-07,
       2.2719495e-15, 1.1752817e-08, 3.0206779e-01, 3.0964156e-07],
      dtype=float32))
(0.9317751, 3, array([3.6300862e-01, 7.6230193e-17, 4.5730890e-06, 9.3177509e-01,
       7.2639756e-09, 5.7133852e-06, 1.2335099e-08, 1.9449960e-10,
       5.3643269e-08, 1.5533676e-06, 2.2637274e-10, 1.6026634e-07,
       7.5502087e-16, 1.4905275e-08, 4.4586863e-03, 1.2229334e-07],
      dtype=float32))
(0.9998996, 2, array([1.2934540e-02, 4.5452753e-06, 9.9989963e-01, 5.5632601e-04

In [ ]:
ims[0]

In [ ]:
model2.save("model2")
!tar -czvf model2.tar.gz model2
!mv model2.tar.gz drive/MyDrive/model4.tar.gz

INFO:tensorflow:Assets written to: model2/assets
model2/
model2/saved_model.pb
model2/assets/
model2/variables/
model2/variables/variables.index
model2/variables/variables.data-00000-of-00001


In [ ]:
model2.save("model2")
model.save("model1")

INFO:tensorflow:Assets written to: model2/assets


INFO:tensorflow:Assets written to: model2/assets


INFO:tensorflow:Assets written to: model1/assets


INFO:tensorflow:Assets written to: model1/assets


In [ ]:
!tar -czvf model2.tar.gz model2
!tar -czvf model1.tar.gz model1

model2/
model2/saved_model.pb
model2/assets/
model2/variables/
model2/variables/variables.index
model2/variables/variables.data-00000-of-00001
model1/
model1/saved_model.pb
model1/assets/
model1/variables/
model1/variables/variables.index
model1/variables/variables.data-00000-of-00001
